# 🔍 Progressive Pipeline Challenge: From Data Quality to Medallion Architecture

**Welcome to the advanced pipeline engineering challenge!** Now that you've seen how the basic pipeline works, it's time to extend it with professional data engineering patterns.

## 📋 Challenge Overview

This challenge has **3 progressive levels** - choose your difficulty or complete all three to master pipeline engineering:

| Level | Name | Difficulty | What You'll Build |
|-------|------|------------|-------------------|
| 🥉 | **Data Quality Guardian** | Intermediate | Add data quality checks to existing tables |
| 🥈 | **Anomaly Detective** | Advanced | Create new anomaly detection table |
| 🥇 | **Pipeline Architect** | Expert | Build complete Bronze→Silver→Gold architecture |

---

## 📖 Background Story

You're a Data Engineer at the wind turbine operations center. The basic pipeline is running, but your team lead says:

> *"The pipeline works, but we need better data quality, automated anomaly detection, and a proper medallion architecture. Can you enhance it?"*

**Your mission:** Extend the pipeline in `01.1-SDP-Wind-Turbine-SQL.ipynb` to make it production-ready!

---

## 🎯 Choose Your Challenge Level

**New to pipelines?** → Start with Level 1  
**Comfortable with SQL?** → Jump to Level 2  
**Want the full experience?** → Complete all 3 levels!

Let's get started!

---

# 🥉 LEVEL 1: Data Quality Guardian

**Difficulty:** Intermediate

## 📚 What You'll Learn

- How to add data quality checks with `CONSTRAINT` and `EXPECT`
- Different violation strategies: `DROP ROW`, `FAIL`, `QUARANTINE`
- Why data quality matters in production pipelines

## 🎯 Your Mission

The operations team reports that sometimes **bad sensor data** causes false alerts. Your task: Add quality checks to prevent invalid data from flowing through the pipeline.

## 📝 Task Instructions

1. Open the pipeline notebook: `01-Data-ingestion/01.1-SDP-Wind-Turbine-SQL.ipynb`
2. Find the `sensor_hourly` table definition
3. Add **3-5 CONSTRAINT statements** to validate the data

### Quality Rules to Implement

Add constraints that check for:

✅ **Power values are realistic**
- Power should be between 0 and 5000 kW
- Action: DROP ROW if violated (bad sensor reading)

✅ **Vibration is within safe limits**
- Vibration should be between 0 and 2
- Action: FAIL if violated (pipeline should stop - critical issue!)

✅ **No null turbine IDs**
- Turbine ID must not be null
- Action: DROP ROW if violated

✅ **Temperature is reasonable**
- Temperature should be between -30 and 50 degrees
- Action: DROP ROW if violated

✅ **Your choice!** Think of another quality rule

## 💡 Code Hints

```sql
-- In 01.1-SDP-Wind-Turbine-SQL.ipynb, find this table:
CREATE OR REFRESH LIVE TABLE sensor_hourly
-- Add constraints like this AFTER the table name:
(
  CONSTRAINT valid_power EXPECT (avg_power > 0 AND avg_power < 5000) ON VIOLATION DROP ROW,
  CONSTRAINT valid_vibration EXPECT (avg_vibration BETWEEN 0 AND 2) ON VIOLATION FAIL,
  -- Add more constraints here...
)
AS SELECT ...
```

### ON VIOLATION Strategies

- **DROP ROW:** Silently remove bad rows (for occasional bad readings)
- **FAIL:** Stop the pipeline (for critical data issues)
- **QUARANTINE:** Move to separate table for investigation

## 📊 Success Criteria

✅ Added at least 3 CONSTRAINT statements  
✅ Used different ON VIOLATION strategies  
✅ Pipeline runs successfully 

**Level 1 Complete!** 🎉 You've added production-grade data quality checks!

---

# 🥈 LEVEL 2: Anomaly Detective

**Difficulty:** Advanced

## 📚 What You'll Learn

- How to create new LIVE TABLEs in a pipeline
- Window functions and statistical calculations
- Z-score methodology for anomaly detection
- Pipeline dependencies with `LIVE.table_name`

## 🎯 Your Mission

Manual anomaly queries are too slow! Your manager wants **automated anomaly detection** that updates in real-time as new sensor data arrives.

**Goal:** Create a new table `turbine_anomaly_scores` that automatically calculates anomaly scores for every turbine.

## 📝 Task Instructions

1. Open `01-Data-ingestion/01.1-SDP-Wind-Turbine-SQL.ipynb`
2. **Add a completely new table** at the end of the pipeline
3. Calculate **Z-scores** for power and vibration sensors
4. Classify turbines as NORMAL, WARNING, or CRITICAL

## 💡 Implementation Guide

### Step 1: Create the Table Structure

Add this to the end of your pipeline notebook:

```sql
CREATE OR REFRESH LIVE TABLE turbine_anomaly_scores
COMMENT "Real-time anomaly detection for turbines using Z-score method"
AS
-- Your code here
```

### Step 2: Calculate Fleet Statistics

Use a CTE (Common Table Expression) to calculate fleet-wide averages:

```sql
WITH fleet_stats AS (
  SELECT 
    AVG(avg_power) as fleet_mean_power,
    STDDEV(avg_power) as fleet_std_power,
    AVG(avg_vibration) as fleet_mean_vibration,
    STDDEV(avg_vibration) as fleet_std_vibration
  FROM LIVE.sensor_hourly  -- Note: LIVE.table_name for pipeline dependencies!
)
```

### Step 3: Calculate Z-Scores

For each turbine, calculate how many standard deviations away from the mean:

```sql
SELECT
  s.turbine_id,
  s.window,
  s.avg_power,
  s.avg_vibration,
  -- Z-score formula: (value - mean) / std_dev
  (s.avg_power - f.fleet_mean_power) / NULLIF(f.fleet_std_power, 0) as power_z_score,
  (s.avg_vibration - f.fleet_mean_vibration) / NULLIF(f.fleet_std_vibration, 0) as vibration_z_score
FROM LIVE.sensor_hourly s
CROSS JOIN fleet_stats f
```

### Step 4: Add Anomaly Classification

Create a status field based on Z-scores:

```sql
CASE
  WHEN ABS(power_z_score) > 3 OR ABS(vibration_z_score) > 3 THEN 'CRITICAL'
  WHEN ABS(power_z_score) > 2 OR ABS(vibration_z_score) > 2 THEN 'WARNING'
  ELSE 'NORMAL'
END as anomaly_level
```

## 🎓 Bonus Challenges (Optional)

### Bonus 1: Multi-Sensor Score
Combine multiple sensors into one anomaly score:
```sql
(ABS(power_z_score) + ABS(vibration_z_score) + ABS(temp_z_score)) / 3 as composite_score
```

### Bonus 2: Business Impact
Calculate estimated revenue loss for anomalous turbines:
```sql
CASE 
  WHEN avg_power < fleet_mean_power 
  THEN (fleet_mean_power - avg_power) * 0.05  -- $0.05 per kW
  ELSE 0 
END as estimated_hourly_revenue_loss
```



## 📊 Success Criteria

✅ Created new `turbine_anomaly_scores` LIVE TABLE  
✅ Calculated Z-scores for at least 2 sensors  
✅ Classified turbines as NORMAL/WARNING/CRITICAL  
✅ Pipeline runs successfully with new table  

**Level 2 Complete!** 🎉 You've built automated anomaly detection!

---

# 🥇 LEVEL 3: Pipeline Architect

**Difficulty:** Expert

## 📚 What You'll Learn

- Medallion Architecture (Bronze → Silver → Gold)
- Layered data processing patterns
- Incremental processing strategies
- Business metrics calculation

## 🎯 Your Mission

Your team lead is impressed! Now they want you to **refactor the entire pipeline** using the Medallion Architecture pattern - the industry standard for data lakes.

**Goal:** Build a 3-layer architecture:
- 🟤 **Bronze:** Raw data with minimal cleaning
- 🥈 **Silver:** Business logic and aggregations  
- 🥇 **Gold:** Analytics-ready tables with business metrics

## 📝 Architecture Overview

```
┌─────────────────────────────────────────────────────────┐
│  BRONZE Layer (Raw + Validated)                        │
│  - sensor_bronze_clean: Deduplicated raw sensor data   │
│  - Quality: Structural validation only                 │
└─────────────────────────────────────────────────────────┘
                         ↓
┌─────────────────────────────────────────────────────────┐
│  SILVER Layer (Business Logic)                         │
│  - sensor_silver_hourly: Hourly aggregations           │
│  - Quality: Business rule validation                   │
└─────────────────────────────────────────────────────────┘
                         ↓
┌─────────────────────────────────────────────────────────┐
│  GOLD Layer (Analytics)                                │
│  - turbine_gold_anomalies: Anomaly detection + scores  │
│  - turbine_gold_kpis: Business metrics & revenue       │
│  - Quality: Analytics validation                       │
└─────────────────────────────────────────────────────────┘
```

## 💡 Implementation Guide

### 🟤 BRONZE LAYER: Clean Raw Data

**Purpose:** Store raw data with minimal transformations, just remove duplicates and enforce structure.

```sql
-- Bronze Layer: Deduplicated sensor data
CREATE OR REFRESH LIVE TABLE sensor_bronze_clean
(
  CONSTRAINT valid_timestamp EXPECT (timestamp IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_turbine_id EXPECT (turbine_id IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "Bronze: Cleaned and deduplicated raw sensor readings"
AS 
SELECT DISTINCT
  turbine_id,
  timestamp,
  AN,
  AVALUES,
  SPEED,
  TORQUE,
  FORCE
FROM LIVE.sensor_bronze
WHERE turbine_id IS NOT NULL;
```

---

### 🥈 SILVER LAYER: Business Logic

**Purpose:** Apply business transformations, aggregations, and enrichments.

```sql
CREATE OR REFRESH LIVE TABLE sensor_silver_hourly
(
  CONSTRAINT power_reasonable EXPECT (avg_power BETWEEN 0 AND 5000) ON VIOLATION DROP ROW,
  CONSTRAINT vibration_safe EXPECT (avg_vibration BETWEEN 0 AND 2) ON VIOLATION FAIL
)
COMMENT "Silver: Hourly sensor aggregations with business rules"
AS
SELECT
  turbine_id,
  window(timestamp, "1 hour") as hour_window,
  AVG(AVALUES) as avg_power,
  MAX(AVALUES) as max_power,
  STDDEV(AVALUES) as power_variability,
  AVG(SPEED) as avg_vibration,
  AVG(TORQUE) as avg_temperature,
  COUNT(*) as reading_count
FROM LIVE.sensor_bronze_clean
GROUP BY turbine_id, window(timestamp, "1 hour");
```

---

### 🥇 GOLD LAYER: Analytics-Ready

**Purpose:** Business-ready tables with KPIs, anomaly scores, and revenue metrics.

```sql
CREATE OR REFRESH LIVE TABLE turbine_gold_anomalies
(
  CONSTRAINT has_anomaly_score EXPECT (power_z_score IS NOT NULL),
  CONSTRAINT valid_level EXPECT (anomaly_level IN ('NORMAL', 'WARNING', 'CRITICAL'))
)
COMMENT "Gold: Real-time anomaly detection with business impact"
AS
WITH fleet_stats AS (
  SELECT 
    AVG(avg_power) as fleet_mean_power,
    STDDEV(avg_power) as fleet_std_power,
    AVG(avg_vibration) as fleet_mean_vibration,
    STDDEV(avg_vibration) as fleet_std_vibration
  FROM LIVE.sensor_silver_hourly
)
SELECT
  s.turbine_id,
  s.hour_window,
  s.avg_power,
  s.avg_vibration,
  
  -- Z-scores
  (s.avg_power - f.fleet_mean_power) / NULLIF(f.fleet_std_power, 0) as power_z_score,
  (s.avg_vibration - f.fleet_mean_vibration) / NULLIF(f.fleet_std_vibration, 0) as vibration_z_score,
  
  -- Anomaly classification
  CASE
    WHEN ABS((s.avg_power - f.fleet_mean_power) / NULLIF(f.fleet_std_power, 0)) > 3 THEN 'CRITICAL'
    WHEN ABS((s.avg_power - f.fleet_mean_power) / NULLIF(f.fleet_std_power, 0)) > 2 THEN 'WARNING'
    ELSE 'NORMAL'
  END as anomaly_level,
  
  -- Business impact: Revenue loss estimation
  CASE 
    WHEN s.avg_power < f.fleet_mean_power 
    THEN (f.fleet_mean_power - s.avg_power) * 0.05
    ELSE 0 
  END as estimated_hourly_revenue_loss
  
FROM LIVE.sensor_silver_hourly s
CROSS JOIN fleet_stats f;
```

---

## 📊 Success Criteria

✅ Created Bronze layer (cleaned data)  
✅ Created Silver layer (aggregations)  
✅ Created Gold layer (business metrics)  
✅ All tables have appropriate constraints  
✅ Can query business KPIs from Gold layer  

**Level 3 Complete!** 🏆 You've built a production-grade Medallion Architecture!